In [ ]:
import numpy as np
import random
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
import statistics

In [ ]:
def process_beta2(beta):
    
    loss=beta.T@beta
    loss=math.sqrt(loss)
    
    return loss

In [ ]:
def get_betaloss(w_gu,beta,P):
    
    loss=0
    sum_loss=0
    for i in range(P):
        loss=(w_gu[i]-beta[i])
        sum_loss=sum_loss+loss*loss
    
    sum_loss=sum_loss
    print(sum_loss)
    return sum_loss

In [ ]:
def process_beta1_2(beta,p,lambda_):
    sum_loss=0
    for i in range(p):
        loss=math.sqrt(abs(beta[i]))
        sum_loss=sum_loss+loss
    return sum_loss

In [ ]:
def get_l1(a,b):
    sum=0
    for i in range(len(a)):
        sum+=abs(a[i]-b[i])
    return sum

In [ ]:
def process_Loss(beta,Y,Z,lambda_):
    m,n,xx=Z.shape
    sum_loss=0
    for i in range(m):
        loss=beta.T@Z[i]@beta-Y[i]
        sum_loss=sum_loss+loss*loss
    
    
    sum_loss=sum_loss+lambda_*process_beta1_2(beta,n,lambda_)
    print(sum_loss)
    
    return sum_loss

In [ ]:
def process_Loss_x(beta,Y,Z,lambda_):
    m,n=Z.shape
    sum_loss=0
    loss=Z@beta-Y
    sum_loss=sum_loss+loss*loss
    
    
    sum_loss=sum_loss+lambda_*process_beta1_2(beta,n,lambda_)
    print(sum_loss)
    
    return sum_loss

In [ ]:
def process_Z(n,p):
    
    matrix_dim = (n, p, p)
    Z=np.empty(matrix_dim)
    
    for i in range(n):
        z=np.random.randn(p,p)
        Z[i]=z
    
    return Z

In [ ]:
def process_beta(sparse,p):
    
    o2=np.zeros(p)
    index=np.random.choice(p,size=sparse,replace=False,p=None)
    
    random_number=[]
    for i in range (sparse):
        random_number.append(random.choice([1, -1]))
    random_number=np.array(random_number) 
    
    o2[index]=random_number
    
    return o2,index

In [ ]:
def process_X(n,p):
    
    X=np.random.randn(n,p)
    
    
    return X

In [ ]:
def process_Y(n,p,Z,X,beta):
    
    Random=np.random.normal(0,1,n)
    
    A_beta=(Z@beta+X)
    matrix_dim = (n, 1)
    Y=np.empty(matrix_dim)
    
    for i in range(n):
        y_i=beta.T@Z[i]@beta+X[i]@beta+Random[i]
        Y[i]=y_i
    
    return Y

In [ ]:
def process_eig(Z,n):
    max_eig=0
    for i in range(n):
        eig, _ = np.linalg.eig(Z[i])
        max_eig = max(np.max(eig),max_eig)
        print(max_eig)
    return max_eig

In [ ]:
def Generate_omega_k(beta,K,h):
    p=len(beta)
    w_k=np.empty((K,p))
    for k in range(K):
        random_number=[]
        for i in range (p):
            random_number.append(random.choice([1, -1]))
        random_number=np.array(random_number)
        w_k[k]=beta+(h/p)*1.0*random_number 
    return w_k

In [ ]:
import numpy as np
import math

def process_features(X):
    m, n = X.shape
    X = np.c_[np.ones((m, 1)), X]
    return X

class l12_IHTA:  
    def __init__(self, Lambda, maxIter=1000):
        self.lambda_ = Lambda
        self.maxIter = maxIter 
        
    def getphi(self,x,u):
        tem=pow(abs(x)/3.0,(-3.0/2))
        o=self.lambda_*u/8.0*tem
        return math.acos(o)
        
        
    def Getf(self,x,alpha):
        return 2/3*x*(1+math.cos(2/3*math.pi-2/3*self.getphi(x,alpha)))
    
        
    def soft_threshold(self,t, B_u,n):
            
        for i in range(n):
            
            s_1=(self.lambda_*t)
            s=pow(54,1/3)/4*pow(s_1,2/3)
            if abs(B_u[i])>s:
                B_u[i]=self.Getf(B_u[i],t)
    
            else :
                B_u[i]=0
                
        
        return B_u
        
    def fit(self, y,Z,alpha_):
        
        
        
        alpha=alpha_
        m,n,xx = Z.shape
        w = np.ones(n)
        z = np.ones(n)
        a=1
        for t in range(self.maxIter):
            sum_=0
            for i in range(m):
                s_1=z.T@Z[i]@z-y[i]
                s_1=s_1*2
                s_2=Z[i]@z
                s_3=s_2
                s_=s_1*s_3
                t_1=z.T@Z[i].T@z-y[i]
                t_1=t_1*2
                t_2=Z[i].T@z
                t_3=t_2
                t_=t_1*t_3
                sum_=sum_+s_+t_
            g=sum_/m
            
            r=z-alpha*g
            sw = w  
            #z=w-alpha*g
            
            w=self.soft_threshold(alpha, r,n)
            p = a
            
            a = (1+math.sqrt(1+4*p*p))/2
            z = w+(p-1)/a*(w-sw)
            
            w_loss=process_Loss(w,y,Z,self.lambda_)

        self.w=w
        
    def predict(self,X):
        return X@(self.w)
    
    def get(self):
        return self.w
    

In [ ]:

import numpy as np
import math

def process_features(X):
    m, n = X.shape
    X = np.c_[np.ones((m, 1)), X]
    return X

class l12_IHTA_x:  
    def __init__(self, Lambda, maxIter=1000):
        self.lambda_ = Lambda
        self.maxIter = maxIter 
        
    def getphi(self,x,u):
        tem=pow(abs(x)/3.0,(-3.0/2))
        o=self.lambda_*u/8.0*tem
        return math.acos(o)
        
        
    def Getf(self,x,alpha):
        return 2/3*x*(1+math.cos(2/3*math.pi-2/3*self.getphi(x,alpha)))
    
        
    def soft_threshold(self,t, B_u,n):
            
        for i in range(n):
            
            s_1=(self.lambda_*t)
            s=pow(54,1/3)/4*pow(s_1,2/3)
            if abs(B_u[i])>s:
                B_u[i]=self.Getf(B_u[i],t)
    
            else :
                B_u[i]=0
                
        
        return B_u
        
    def fit(self, y,Z,alpha_):
        
        alpha=alpha_
        m,n = Z.shape
        w = np.zeros((n,1))
        z = np.zeros((n,1))
        a=1
        for t in range(self.maxIter):
            g1=(Z@z-y)
            g2=Z.T@g1
            g=g2*2
            
            r=z-alpha*g
            
            sw = w  
         
            
            w=self.soft_threshold(alpha, r,n)
            p = a
            
            a = (1+math.sqrt(1+4*p*p))/2
            z = w+(p-1)/a*(w-sw)
            
            w_loss=process_Loss_x(w,y,Z,self.lambda_)
            
        self.w=w
        
    def predict(self,X):
        return X@(self.w)
    
    def get(self):
        return self.w
    

In [ ]:
n=50
p=500

sparse=450
beta,indice=process_beta(sparse,p)
X=process_X(n*3,p)
Z=process_Z(n*3,p)
Y=process_Y(n*3,p,Z,X,beta)
print(Y.shape)


(150, 1)


In [ ]:
model = l12_IHTA(Lambda=0.4, maxIter=1000)
model.fit(Y,Z,1/100000)
w_true=model.get()

[58117678.1367858]
[49431579.9150186]
[41048099.11532496]
[33535922.18573856]
[27088240.69013795]
[21688617.63839949]
[17226805.0545413]
[13566959.01050436]
[10580990.52441962]
[8160374.0739047]
[6216542.47009286]
[4676502.99420082]
[3477635.22826058]
[2563697.33346306]
[1882865.27993265]
[1387316.72541939]
[1034010.85925688]
[785738.83290951]
[611873.23456136]
[488537.71233426]
[398218.16371093]
[328937.41018947]
[273182.92235016]
[226758.34896828]
[187672.26818095]
[155178.48045835]
[129020.9442397]
[108924.94876963]
[94330.92671395]
[84334.81497541]
[77777.64822552]
[73413.77412556]
[70095.72208529]
[66924.14513322]
[63330.71458667]
[59085.7996863]
[54242.09604544]
[49038.09837207]
[43789.78946003]
[38796.96243047]
[34276.02436141]
[30336.48729159]
[26983.6103248]
[24146.13941662]
[21712.78051269]
[19567.93092158]
[17615.96994073]
[15795.82488092]
[14082.6025004]
[12480.63970181]
[11011.96765828]
[9705.18406306]
[8584.2332657]
[7661.64702442]
[6936.23463871]
[6393.27766967]
[6006.95

KeyboardInterrupt: 

In [ ]:
print(model.lambda_)